In [1]:
import datetime
import numpy as np
import pandas as pd

from tqdm import tqdm, trange

from airsim.collections import AirObject, AirEnv, RadarSystem, ControlPoint, Supervisor
from airsim.time import Time

In [2]:
def save_data(df, name, train=True, timestamp=True):
    path = "data/"

    if train:
        path = path + "train"
    else:
        path = path + "test"

    path = path + "/" + name
    
    if timestamp:
        ts = "_" + datetime.datetime.now().strftime("%d%m%H%M%S")
        path = path + ts

    path = path + ".csv"

    print(f"Saving to {path}")
    df.to_csv(path, index=False)

In [3]:
t_min = 0
t_max = 1000000
dt = 10

In [4]:
def f(t):
    return np.array([-130000 + 0.26*t, 4970 * np.arctan(t / 500), 10000])

ao_1 = AirObject(track=lambda t: np.array([-130000 + 0.26*t,  5000, 10000]))
ao_2 = AirObject(track=lambda t: np.array([-130000 + 0.26*t, -5000, 10000]))
ao_3 = AirObject(track=f)

In [5]:
ae = AirEnv(air_objects=[ao_1, ao_2, ao_3])

In [6]:
rs_1 = RadarSystem(position=np.array([     0, 0, 0]), detection_radius=10000 * 26**0.5, error=2.5)
rs_2 = RadarSystem(position=np.array([ 80000, 0, 0]), detection_radius=10000 * 26**0.5, error=2.5)
rs_3 = RadarSystem(position=np.array([-80000, 0, 0]), detection_radius=10000 * 26**0.5, error=2.5)

for rs in (rs_1, rs_2, rs_3):
    rs.set_air_environment(ae)

In [7]:
cp = ControlPoint(radar_systems=[rs_1, rs_2, rs_3])

In [8]:
t = Time()
t.set(t_min)

progressbar = tqdm(range(t_min, t_max + 1, dt), ncols=100)
progressbar.set_description('Running system')
for _ in progressbar:
    if t.get() % 100 == 0:
        rs_1.trigger()
    if t.get() % 100 == 20:
        rs_2.trigger()
    if t.get() % 100 == 80:
        rs_3.trigger()
    t.step(dt)

for rs in (rs_1, rs_2, rs_3):
    rs.estimate_velocity()
    rs.estimate_acceleration()
cp.upload_data()

Running system: 100%|██████████████████████████████████████| 100001/100001 [01:45<00:00, 950.14it/s]


In [9]:
data = cp.get_data()

In [10]:
data

,rs_id,id,time,x,y,z,x_err,y_err,z_err,v_x_est,v_y_est,v_z_est,a_x_est,a_y_est,a_z_est,load_time
0,0,0,308700,-49735.897168,5000.007137,9999.210457,2.5,2.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,1000010
1,0,1,308700,-49735.968443,-5001.572090,10002.225767,2.5,2.5,2.5,NaN,NaN,NaN,NaN,NaN,NaN,1000010
2,0,0,308800,-49712.764013,4999.699788,9997.815229,2.5,2.5,2.5,0.231332,-0.003073,-0.013952,NaN,NaN,NaN,1000010
3,0,1,308800,-49711.337107,-4999.652053,10000.204831,2.5,2.5,2.5,0.246313,0.019200,-0.020209,NaN,NaN,NaN,1000010
4,0,0,308900,-49688.075680,5001.606689,10000.533971,2.5,2.5,2.5,0.246883,0.019069,0.027187,0.000156,0.000221,0.000411,1000010
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34366,2,1,383380,-30323.470754,-5000.572400,9999.262095,2.5,2.5,2.5,0.222732,-0.027358,-0.013056,-0.000539,-0.000475,-0.000361,1000010
34367,2,0,383480,-30296.590555,4998.663559,9998.447866,2.5,2.5,2.5,0.237296,-0.006626,-0.005702,-0.000134,-0.000147,0.000150,1000010
34368,2,1,383480,-30295.896567,-4998.969835,9998.581512,2.5,2.5,2.5,0.275742,0.016026,-0.006806,0.000530,0.000434,0.000063,1000010
34369,2,0,383580,-30270.450414,5002.106701,10000.926159,2.5,2.5,2.5,0.261401,0.034431,0.024783,0.000241,0.000411,0.000305,1000010


In [12]:
save_data(data, '3ao_3rs', train=True, timestamp=True)

Saving to data/train/3ao_3rs_1003223723.csv


In [11]:
assert 0 == 1

AssertionError: 

In [ ]:
class DataGenerator:

    def __init__(self):
        pass

    def generate_1ao_2rs(self, ao_track, t_min, t_max, dt):
        supervisor = Supervisor(
            air_objects=[
                AirObject(track=ao_track)
            ],
            radar_systems=[
                RadarSystem(position=np.array([0, 0, 0]), detection_radius=1e+308, error=0.1),
                RadarSystem(position=np.array([0, 0, 0]), detection_radius=1e+308, error=0.1)
            ]
        )
        supervisor.run(t_min=t_min, t_max=t_max, dt=dt)
        return supervisor.get_data()
        # return self.__get_past_new_detections_cross_join(supervisor.get_data())

    def generate_Nao_2rs(self, ao_tracks, t_min, t_max, dt):
        supervisor = Supervisor(
            air_objects=[
                AirObject(track=ao_track) for ao_track in ao_tracks
            ],
            radar_systems=[
                RadarSystem(position=np.array([0, 0, 0]), detection_radius=1e+308, error=0.1),
                RadarSystem(position=np.array([0, 0, 0]), detection_radius=1e+308, error=0.1)
            ]
        )
        supervisor.run(t_min=t_min, t_max=t_max, dt=dt)
        return supervisor.get_data()
        # return self.__get_past_new_detections_cross_join(supervisor.get_data())

    # def __get_past_new_detections_cross_join_by_time(self, data, t):
    #     detection_columns = ['time', 'x', 'y', 'z', 'x_err', 'y_err', 'z_err']
    #     v_est_columns = [f'v_{axis}_est' for axis in ('x', 'y', 'z')]
    #     a_est_columns = [f'a_{axis}_est' for axis in ('x', 'y', 'z')]

    #     suffix = '_'
    #     cols = detection_columns + v_est_columns + a_est_columns
    #     cols_new = [f'{el}{suffix}' for el in (detection_columns + v_est_columns + a_est_columns)]
    #     dfl = data.loc[data['time'] < t].copy()
    #     dfl = dfl.sort_values(by=['id', 'time', 'err_ratio'], ascending=[True, False, False])
    #     dfl = dfl.drop_duplicates(subset=['id', 'time'])
    #     dfl = dfl.groupby(by=['id']).head(1)
    #     dfl = dfl.rename(columns={cols[i]: cols_new[i] for i in range(len(cols))})
    #     dfl = dfl[['id'] + cols_new]
    #     dfl = dfl.reset_index(drop=True)
        
    #     dfr = data.loc[data['time'] == t].copy()
    #     dfr = dfr.sort_values(by=['id', 'err_ratio'], ascending=[True, False])
    #     dfr = dfr[['id'] + detection_columns]
    #     dfr = dfr.reset_index(drop=True)
        
    #     df = pd.merge(dfl, dfr, how='cross')
    #     df['is_identical'] = df['id_x'] == df['id_y']
    #     df = df.astype({'is_identical': 'float64'})
    #     df = df.drop(columns=['id_x', 'id_y'])
        
    #     return df

    # def __get_past_new_detections_cross_join(self, data):
    #     dfs = []
    #     data['err_ratio'] = np.sqrt(3.0 / (data['x_err'] ** 2 + data['y_err'] ** 2 + data['z_err'] ** 2))

    #     timestamps = sorted(set(data['time']))[1:]
    #     progressbar = tqdm(timestamps, ncols=100)
    #     progressbar.set_description('Generating data')
    #     for t in progressbar:
    #         df_t = self.__get_past_new_detections_cross_join_by_time(data, t)
    #         dfs.append(df_t)

    #     df = pd.concat(dfs).reset_index(drop=True)
    #     return df

In [ ]:
dg = DataGenerator()

In [ ]:
df = dg.generate_Nao_2rs(ao_tracks=[
    lambda t: np.array([5000.0 * np.sin(2*np.pi*t/10000), 1000, 10000]),
    lambda t: np.array([5000.0 * np.sin(2*np.pi*t/10000), 0, 10000])
], t_min=0, t_max=60000, dt=10)

In [ ]:
# save_data(df, '2ao_2rs', train=True, timestamp=True)

In [ ]:
df = dg.generate_Nao_2rs(ao_tracks=[
    lambda t: np.array([0.5 * t, 0, 10000]),
    lambda t: np.array([1.5 * t, 1000, 10000])
], t_min=0, t_max=10000, dt=10)

In [ ]:
# save_data(df, '2ao_2rs', train=False, timestamp=True)